<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-27 11:45:00
-------------------
qualified stocks: 94
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  1.87 L
Current:  1.29 C
-------------------
Today PnL: 51.41 K (0.4%)
Current PnL: -16.26 L (-11.53%)
CY Booked + Current PnL: -11.96 L (-8.48%)
-------------------
Total profit:  3.89 L
Total loss:  -20.14 L
-------------------
Total Booked + Current PnL: 15.02 L (13.17%)
Total Booked PnL: 31.27 L (27.42%)
Curr Year Booked PnL: 4.29 L (3.31%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.09 C
Est FTT PnL: 79.24 L (61.19%)
-------------------
Est LTT:  2.70 C
Est LTT PnL: 1.41 C (108.53%)
Deployed:  1.14 C
Current:  1.29 C
CAGR/XIRR %: 9.01%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SBILIFE,1496.49,1924.99,2.11,H-LC,47.87,101170.0,17367.0,6627.0,0.28,20.72,6.55,28.63,42.0,2.62,0.79,35.19,AR,ATH,INSURANCE
62,RELIANCE,1291.83,1526.00,4.98,X-LC,50.00,155924.0,15115.0,10416.0,-0.30,10.73,6.68,18.13,17.0,1.45,1.22,23.12,XY25,NTT,REFINERIES
4,ADANIPORTS,1103.69,1583.00,-0.04,M-LC,90.43,100280.0,21918.0,12114.0,0.87,27.97,12.08,43.43,71.0,1.81,0.79,34.12,XY24,NTT,MISC
28,GILLETTE,8109.44,10330.69,8.65,H-SC,88.30,160310.0,22450.0,15310.0,0.44,16.28,9.55,27.39,118.0,1.47,1.26,38.94,X40,ATH,FMCG
56,PIDILITIND,2504.06,3576.00,-18.22,X-LC,19.15,90624.0,15502.0,16657.0,-0.80,20.64,18.38,42.81,11.0,0.93,0.71,13.65,X40,BTT,CHEMICALS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,VALIANTORG,512.64,838.00,-354.52,H-SC,97.87,108069.0,-25217.0,109809.0,4.99,-18.92,101.61,63.47,131.0,-0.23,0.85,81.15,XR,NTT,CHEMICALS
9,AWL,342.88,485.00,-68.17,H-MC,8.51,172212.0,-45517.0,135755.0,4.81,-20.91,78.83,41.45,86.0,-0.34,1.35,15.04,XY24,NTT,FMCG
78,TANLA,917.30,1963.11,-37.00,H-SC,98.94,154695.0,-84720.0,357670.0,3.74,-35.39,231.21,114.01,110.0,-0.24,1.21,41.88,AR,ATH,IT
10,BANDHANBNK,214.25,400.00,-33.73,H-SC,43.62,197490.0,-51040.0,266513.0,2.80,-20.54,134.95,86.70,117.0,-0.19,1.55,30.65,XY24,NTT,BANKS
40,INDUSINDBK,1354.50,1800.00,-36.29,H-MC,17.02,93640.0,-60773.0,111563.0,2.66,-39.36,119.14,32.89,91.0,-0.54,0.73,28.94,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VAIBHAVGBL,350.21,521.00,65.84,X-SC,81.91,110535.0,-41456.0,115575.0,-2.14,-27.28,104.56,48.77,59.0,-0.36,0.87,26.96,XR,NTT,JEWELLERY
3,ACC,2579.05,3906.00,-34.60,M-MC,26.60,141277.0,-46994.0,143862.0,-1.18,-24.96,101.83,51.45,153.0,-0.33,1.11,6.98,XY24,BTT,CEMENT
38,INDIAMART,2327.09,4911.36,-52.96,H-SC,40.43,122308.0,-1028.0,137988.0,-1.05,-0.83,112.82,111.05,108.0,-0.01,0.96,19.84,AR,ATH,MISC
66,SAMMAANCAP,170.35,326.00,-160.64,M-SC,12.77,73236.0,-28974.0,122363.0,-0.94,-28.35,167.08,91.37,183.0,-0.24,0.57,18.89,XY25,NTT,FINANCE
56,PIDILITIND,2504.06,3576.00,-18.22,X-LC,19.15,90624.0,15502.0,16657.0,-0.80,20.64,18.38,42.81,11.0,0.93,0.71,13.65,X40,BTT,CHEMICALS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1450.31,2275.00,-12.06,X-LC,58.51,250314.0,19715.0,111415.0,-0.39,8.55,44.51,56.86,1.0,0.18,1.96,14.25,X40,BTT,IT
81,TCS,3841.06,4998.00,-15.52,X-LC,38.30,243487.0,-21546.0,101364.0,-0.24,-8.13,41.63,30.12,2.0,-0.21,1.91,9.20,X40,BTT,IT
50,LTIM,5564.16,7197.33,-4.82,X-LC,86.17,183128.0,-17182.0,75980.0,-0.48,-8.58,41.49,29.35,4.0,-0.23,1.43,26.51,X200,ATH,IT
72,SIEMENS,4406.90,7969.85,-9.47,X-LC,80.85,81925.0,-28247.0,117325.0,0.87,-25.64,143.21,80.85,5.0,-0.24,0.64,20.66,AR,ATH,ELECTRICAL
32,HAVELLS,1593.67,2069.16,-5.33,X-LC,10.64,213438.0,-3301.0,67959.0,0.13,-1.52,31.84,29.84,6.0,-0.05,1.67,11.96,X40,ATH,ELECTRICAL


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,5.78,H-SC,100.00,122293.0,24646.0,23028.0,2.59,25.24,18.83,48.82,138.0,1.07,0.96,35.86,X40,ATH,PHARMA
4,ADANIPORTS,1103.69,1583.00,-0.04,M-LC,90.43,100280.0,21918.0,12114.0,0.87,27.97,12.08,43.43,71.0,1.81,0.79,34.12,XY24,NTT,MISC
68,SBILIFE,1496.49,1924.99,2.11,H-LC,47.87,101170.0,17367.0,6627.0,0.28,20.72,6.55,28.63,42.0,2.62,0.79,35.19,AR,ATH,INSURANCE
56,PIDILITIND,2504.06,3576.00,-18.22,X-LC,19.15,90624.0,15502.0,16657.0,-0.80,20.64,18.38,42.81,11.0,0.93,0.71,13.65,X40,BTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,39.47,M-SC,70.21,144866.0,7288.0,87122.0,-0.42,5.30,60.14,68.62,182.0,0.08,1.14,25.59,X40N,NTT,TEXTILES
28,GILLETTE,8109.44,10330.69,8.65,H-SC,88.30,160310.0,22450.0,15310.0,0.44,16.28,9.55,27.39,118.0,1.47,1.26,38.94,X40,ATH,FMCG
57,POLYCAB,5005.95,7541.35,7.45,H-MC,62.77,101958.0,16857.0,26244.0,0.81,19.81,25.74,50.65,92.0,0.64,0.80,28.16,X40N,ATH,CABLES
55,PFIZER,4245.50,6318.30,5.78,H-SC,100.00,122293.0,24646.0,23028.0,2.59,25.24,18.83,48.82,138.0,1.07,0.96,35.86,X40,ATH,PHARMA
4,ADANIPORTS,1103.69,1583.00,-0.04,M-LC,90.43,100280.0,21918.0,12114.0,0.87,27.97,12.08,43.43,71.0,1.81,0.79,34.12,XY24,NTT,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,-0.44,H-SC,32.98,111983.0,9979.0,35039.0,0.55,9.78,31.29,44.14,111.0,0.28,0.88,27.61,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-31.87,M-MC,18.09,133476.0,4324.0,82742.0,-0.02,3.35,61.99,67.42,155.0,0.05,1.05,29.82,XR,NTT,BANKS
93,WIPRO,243.46,420.00,-9.71,M-LC,46.81,154994.0,4049.0,105411.0,-0.10,2.68,68.01,72.51,69.0,0.04,1.21,16.70,XR,NTT,IT
33,HEROMOTOCO,4311.81,6039.03,-0.48,H-MC,78.72,151774.0,861.0,59586.0,-0.53,0.57,39.26,40.06,89.0,0.01,1.19,24.14,AR,ATH,AUTO
38,INDIAMART,2327.09,4911.36,-52.96,H-SC,40.43,122308.0,-1028.0,137988.0,-1.05,-0.83,112.82,111.05,108.0,-0.01,0.96,19.84,AR,ATH,MISC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TRIDENT,37.35,48.00,68.21,M-SC,93.62,80744.0,-11510.0,37812.0,0.18,-12.48,46.83,28.51,220.0,-0.30,0.63,38.17,XR,NTT,TEXTILES
52,MASFIN,326.60,399.50,-22.76,H-SC,61.70,87690.0,-10290.0,32156.0,0.09,-10.50,36.67,22.32,130.0,-0.32,0.69,27.18,XR,ATH,FINANCE
68,SBILIFE,1496.49,1924.99,2.11,H-LC,47.87,101170.0,17367.0,6627.0,0.28,20.72,6.55,28.63,42.0,2.62,0.79,35.19,AR,ATH,INSURANCE
4,ADANIPORTS,1103.69,1583.00,-0.04,M-LC,90.43,100280.0,21918.0,12114.0,0.87,27.97,12.08,43.43,71.0,1.81,0.79,34.12,XY24,NTT,MISC
88,VALIANTORG,512.64,838.00,-354.52,H-SC,97.87,108069.0,-25217.0,109809.0,4.99,-18.92,101.61,63.47,131.0,-0.23,0.85,81.15,XR,NTT,CHEMICALS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,5.78,H-SC,100.00,122293.0,24646.0,23028.0,2.59,25.24,18.83,48.82,138.0,1.07,0.96,35.86,X40,ATH,PHARMA
78,TANLA,917.30,1963.11,-37.00,H-SC,98.94,154695.0,-84720.0,357670.0,3.74,-35.39,231.21,114.01,110.0,-0.24,1.21,41.88,AR,ATH,IT
88,VALIANTORG,512.64,838.00,-354.52,H-SC,97.87,108069.0,-25217.0,109809.0,4.99,-18.92,101.61,63.47,131.0,-0.23,0.85,81.15,XR,NTT,CHEMICALS
18,CERA,8421.60,10420.45,-17.66,X-SC,96.81,84948.0,-24533.0,50519.0,0.50,-22.41,59.47,23.73,64.0,-0.49,0.67,28.57,X40N,ATH,CERAMICS
7,ASIANTILES,75.41,137.00,5877.78,M-SC,95.74,65609.0,-28201.0,104817.0,-0.68,-30.06,159.76,81.67,203.0,-0.27,0.51,27.05,XR,NTT,CERAMICS


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.39
1,25,40.34
2,50,69.38


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    32.28
XY25    14.35
X40     14.24
X40N    13.67
AR      12.54
XR      10.36
X200     1.43
SR       1.13
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.45
M-SC    17.99
X-LC    17.38
H-MC     8.75
M-LC     7.76
H-LC     7.25
X-MC     5.66
M-MC     5.62
X-SC     2.88
L-SC     2.30
L-LC     1.16
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.21,0.32,30.96
IT,9.99,-17.35,81.49
BANKS,9.66,-13.67,65.93
MISC,6.52,-30.05,87.57
INSURANCE,6.01,2.22,26.94
FINANCE,5.95,-25.85,85.61
PAINTS,5.47,-17.98,52.90
ELECTRICAL,4.80,-4.54,56.58
AUTO,4.50,-8.84,56.38


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2856348.0,26
AR,1243414.0,13
X40N,1203632.0,17
XR,1128450.0,14
X40,743915.0,11
XY25,471246.0,10
SR,200708.0,2
X200,75980.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2282996.0,21
M-SC,2015083.0,21
X-LC,758655.0,12
M-MC,551678.0,5
H-MC,511170.0,8
X-SC,342048.0,4
M-LC,315155.0,6
X-MC,314092.0,5
L-SC,304233.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      920277.0      6
M-SC       XY24      742876.0      6
H-SC       AR        555791.0      3
           X40N      503909.0      6
M-SC       X40N      464371.0      6
X-LC       X40       427007.0      5
M-MC       XY24      388258.0      3
M-SC       XR        292299.0      4
H-SC       XR        264681.0      4
H-LC       AR        249100.0      4
M-SC       X40       214914.0      2
M-LC       XY24      192843.0      4
H-MC       XY24      176844.0      2
L-SC       XR        156179.0      2
X-LC       AR        141294.0      2
X-MC       XY24      140814.0      2
M-SC       XY25      122363.0      1
X-SC       XR        115575.0      1
           SR        115349.0      1
X-LC       XY25      114374.0      4
H-MC       XR        111563.0      1
X-SC       X40N      111124.0      2
M-LC       XR        105411.0      1
X-MC       X40N       97984.0      2
M-SC       AR         92901.0      1
           SR         85359.0      1
L-SC       XY24       83990.0      1
M-MC       XR         82742.0      1
           AR         80678.0      1
X-LC       X200       75980.0      1
X-MC       XY25       75294.0      1
H-MC       XY25       73277.0      1
L-SC       AR         64064.0      1
H-MC       X40        63656.0      2
           AR         59586.0      1
L-LC       XY25       41010.0      1
H-SC       X40        38338.0      2
H-LC       XY25       28027.0      1
H-MC       X40N       26244.0      1
M-LC       XY25       16901.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
